In [10]:
import openai
import json
from pathlib import Path
client = openai.Client()

In [99]:
big_context = True
big_list_path = Path("data/movie-list.csv")
if not big_list_path.exists():
    !wget -O data/popular-movies-corpora.json https://raw.githubusercontent.com/dariusk/corpora/master/data/film-tv/popular-movies.json
    big_list = json.loads(Path("data/popular-movies-corpora.json").read_text())['popular-movies']
    big_list_path.write_text("\n".join(big_list))

big_list = big_list_path.read_text().split("\n")
# load the list, and potentially expand it
big_list_md = "- " + "\n- ".join(big_list)
prompt = 'Continue the following list with 200 new items. List individual movies (e.g. don\'t group a series into one entry.): \n\n' + big_list_md

msgs = [
    {'role':'system', 'content': 'You automate movie lists.'},
    {'role':'user', 'content': prompt}
]
response = client.chat.completions.create(
    model='gpt-3.5-turbo-16k' if big_context else 'gpt-3.5-turbo',
    messages=msgs
    )
completion = response.choices[0].message.content
new_movies = [m[2:].strip() for m in completion.split('\n') if m.startswith('- ')]
assert response.choices[0].finish_reason != 'length', f"The response was cut off for length! Check the outputs (last output is `{new_movies[-1]}`)"

In [102]:
# check new data
shuffle(new_movies)
new_movies[:20]

['Avengers Confidential: Black Widow & Punisher (2014)',
 'Captain Marvel (2019)',
 'The Suicide Squad (2021)',
 'X-Men: The Last Stand (2006)',
 'Spider-Man: Homecoming (2017)',
 'Star Wars: Episode IV - A New Hope (1977)',
 "Pirates of the Caribbean: Dead Man's Chest (2006)",
 'Scott Pilgrim vs. The World (2010)',
 'Doctor Strange (2016)',
 'X2: X-Men United (2003)',
 'Underworld (2003)',
 'Black Widow',
 'Justice League (2017)',
 'Fantastic Beasts: The Crimes of Grindelwald (2018)',
 'Black Widow (2021)',
 'Fast & Furious 6 (2013)',
 'The Matrix (1999)',
 'Fant4stic (2015)',
 'Doctor Strange (2016)',
 'Jurassic World: Dominion (2022)']

In [103]:
# add to big list, dedupe, shuffle, and save
from random import shuffle

big_list += new_movies
big_list = list(set(big_list))
shuffle(big_list)
print('N: ', len(big_list))
big_list_path.write_text("\n".join(big_list))


N:  1066


26819